In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

import scsavailability as scs
    
import statsmodels.api as sm
from scipy import stats

from scsavailability import features as feat, model as md, plotting as pt, score as sc


2020-12-14 13:22:43,644 - NumExpr defaulting to 8 threads.


In [3]:
path = 'C:/Users/Jamie.williams/OneDrive - Newton Europe Ltd/Castle Donnington/Data/'

In [5]:
at = pd.read_csv(path+'active_totes_20201123.csv')
av = pd.read_csv(path+'Availability_with_Grey&Blue_1811-2511.csv')
#fa = pd.read_csv(path + 'scs_tote_matched.csv')
scs_raw = pd.read_csv(path + 'SCS_Alerts_NOV_new.csv')



In [6]:
fa = feat.add_code(scs_raw)
fa, unmapped = feat.add_tote_colour(fa)

Running: Function "load_tote_lookup" (id=2486671000192) was called 1 times


In [7]:
at = feat.pre_process_AT(at)

Running: Function "pre_process_AT" (id=2486671001344) was called 1 times


In [8]:
av = feat.pre_process_av(av)

Running: Function "pre_process_av" (id=2486671003072) was called 1 times


In [9]:
fa = feat.preprocess_faults(fa,remove_same_location_faults = True)

Running: Function "preprocess_faults" (id=2486671003360) was called 1 times
Running: Function "load_module_lookup" (id=2486671001920) was called 1 times
duplicated location faults removed - max duration kept


In [50]:
fa_floor = feat.floor_shift_time_fa(fa, shift=0)

Time shifted by 0Minutes


In [79]:
fa_sel = feat.fault_select(fa_floor, fault_select_options = {'Tote Colour':['Grey','Both']})
                                                            
fa_agg = feat.faults_aggregate(fa_sel,fault_agg_level= 'code', agg_type = 'count')

Running: Function "faults_aggregate" (id=2486671106544) was called 9 times


In [89]:
av_sel,at_sel = feat.av_at_select(av, at, remove_high_AT = False,availability_select_options = None)

av_agg = feat.aggregate_availability(av_sel, agg_level = 'Module')
at_agg = feat.aggregate_totes(at_sel, agg_level = 'Module')

#at_agg = at_agg[at_agg['TOTES']>100]

Running: Function "aggregate_availability" (id=2486671106976) was called 18 times


In [90]:
df = feat.merge_av_fa_at(av_agg ,at_df=at_agg, fa_df = fa_agg, target = 'Grey Tote Loss' ,faults=True, totes = True, agg_level = 'Module',remove_0=True)

Datasets merged


In [85]:
X,y = md.gen_feat_var(df)
y=1-y

In [86]:
X_train, X_test, y_train, y_test = md.split(X,y,split_options = {'test_size': 0.3,
                                                                 'random_state': None})

RF_mdl,predictions_RF=md.run_RF_model(X_train, X_test, y_train, y_test,RF_options = {'num_trees': 100, 
                                                                                    'criterion': 'mse', 
                                                                                    'max_depth': None, 
                                                                                    'dtree': False})

Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

XGB_mdl,predictions_XGB,Importance_XGB = md.run_XGB_model(X_train, X_test, y_train, y_test, XGB_options = {'num_trees': 100,
                                                                                                         'max_depth': None})

selection_options= {'thres': -np.inf,
                    'max_feat': None}

X_sel_rf = md.select_features(X, y, RF_mdl, selection_options)

X_sel_lm = md.select_features(X, y, Linear_mdl, selection_options)

X_sel_XGB = md.select_features(X, y, XGB_mdl, selection_options)

                                                                                                           #X_train, X_test, y_train, y_test = md.split(X_sel_lm,y,test_size=0.3,random_state=101)

#Linear_mdl,predictions_LM,Coeff,fit_metrics = md.run_LR_model(X_train, X_test, y_train, y_test)

cv_R2 = md.cross_validate_r2(Linear_mdl, X_sel_lm, y)

cv_R2 = md.cross_validate_r2(RF_mdl,X_sel_rf, y)

cv_R2 = md.cross_validate_r2(XGB_mdl, X_sel_XGB, y)

#md.stats_model(Linear_mdl,X_test,y_test)

Feature Importance Ranking: 
 
    Importance      Feature
0    0.262254        TOTES
1    0.061540  C1402STA061
2    0.032321       SCSM15
3    0.031531          C05
4    0.027877       SCSM18
5    0.027742  C1404STA164
6    0.021530       SCSM02
7    0.019732  C1102STA077
8    0.019602       SCSM03
9    0.018359  C1404STA197

Regression Metrics: 
 
           RF Metrics
MAE         0.151031
MSE         0.048972
RMSE        0.221296
MAPE%     294.270000
ACC%     -194.270000
OOB        -0.052970
R2_Train    0.853113
R2_Pred    -0.150503 

Feature Importance Ranking: 
 
    Importance      Feature
0    0.097398  C1404STA174
1    0.096304  C1402STA061
2    0.091202       SCSM15
3    0.083370  C0802STA065
4    0.067933  C1102STA077
5    0.050986  C1104STA177
6    0.041970  C1104STA197
7    0.032817  C0502STA097
8    0.030666  C0602STA097
9    0.029800  C1002STA077

Regression Metrics: 
 
          XGB Metrics
MAE         0.169537
MSE        0.0552803
RMSE        0.235118
MAPE%         285

### Add LM - metrics

In [27]:
sc.lm_coefficients(Linear_mdl, X, y)

LinAlgError: Singular matrix

In [18]:
X_train = sm.add_constant(X_train)
model = sm.OLS(y_train,X_train)
results = model.fit()
print(results.summary())

X_new = sm.add_constant(X_test)
#y_prediction =  results.predict(X_test)

# plt.scatter(y_prediction,y_test)

# from sklearn.metrics import r2_score
# print(r2_score(y_prediction,y_test))

                            OLS Regression Results                            
Dep. Variable:         Blue Tote Loss   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.824
Method:                 Least Squares   F-statistic:                     193.3
Date:                Mon, 14 Dec 2020   Prob (F-statistic):           2.31e-31
Time:                        13:25:04   Log-Likelihood:                 141.41
No. Observations:                  83   AIC:                            -276.8
Df Residuals:                      80   BIC:                            -269.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.2636      0.032      8.349   

In [75]:
df_greyhigh_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_ID.sum().sort_values(ascending=False).head(20)

3611    9364
5105    3500
3504    2534
3416     997
5120     790
3419     754
5032     716
5329     650
5608     444
5416     373
5033     278
5775     271
5768     271
5583     210
5055     204
5576     200
5054     178
3420     174
5344     162
5774     148
dtype: int64

In [78]:
df_grey_ID = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_ID.sum().sort_values(ascending=False).head(20)

3611    130133
5105     47816
3504     34538
3416     12717
5120     11698
3419     10039
5329      9735
5032      9324
5608      5345
5416      5075
5033      4608
5768      3562
5775      3348
5576      2970
5583      2844
5055      2735
3420      2232
5100      2230
5054      2214
5582      2177
dtype: int64

In [88]:
df_greyhigh_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_greyhigh_Asset.sum().sort_values(ascending=False).head(20)

SCSM12         1715
C09            1580
C12            1520
SCSM18         1412
C14            1392
C08            1321
C13             950
SCSM03          890
SCSM15          751
C07             701
C11             630
C06             619
SCSM07          615
SCSM11          610
C1404STA164     606
SCSM01          601
SCSM04          586
SCSM13          575
SCSM17          543
SCSM19          508
dtype: int64

In [91]:
df_grey_Asset = df.drop(['Grey Tote Loss','TOTES','timestamp'],axis=1)

df_grey_Asset.sum().sort_values(ascending=False).head(20)

C09            21951
SCSM18         21901
SCSM12         20747
C12            20675
C14            17935
C08            17413
C13            12948
SCSM03         11746
SCSM15         10073
C07             9986
C11             9786
C06             9391
SCSM11          8923
SCSM01          8745
SCSM07          8369
SCSM04          8079
C1404STA164     7919
SCSM13          7502
SCSM20          7361
SCSM19          6854
dtype: int64

In [93]:
scs_raw

,Number,Alert,Entry time,PLC,Desk,Duration,Fault ID
0,141254,C17: C1701STA254 254B12 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:13,3416
1,138183,C17: C1701STA012 012B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:03,3416
2,122518,C15: C1502STA034 034B12 occupancy sensor fault,01/11/2020 00:00,C15,P02,0.00:00:05,3416
3,444609,SCS M09: single module 1 emergency stop unit ...,01/11/2020 00:00,SCSM09,SCSM09,0.00:00:13,10408
4,139815,C17: C1701STA112 112B13 occupancy sensor fault,01/11/2020 00:00,C17,P01,0.00:00:08,3416
...,...,...,...,...,...,...,...
174085,439314,SCS228: sto: lifter occupation wrong in auto-m...,25/11/2020 09:03,SCSM22,SCS228,0.00:00:04,5113
174086,439314,SCS228: sto: lifter occupation wrong in auto-m...,25/11/2020 09:03,SCSM22,SCS228,0.00:00:08,5113
174087,439314,SCS228: sto: lifter occupation wrong in auto-m...,25/11/2020 09:04,SCSM22,SCS228,0.00:00:22,5113
174088,439308,SCS228: sto: locking device timeout close X:45...,25/11/2020 03:15,SCSM22,SCS228,0.00:03:29,5107
